# (Ignore) Calculate the 10 1-octave bands and the 3 SPL peaks with corresponding frequencies (based on the preprocessed, but non-normalized data)

I don't know how smart it is to mix non-normalized with normalized approaches. Maybe if we normalize the features before training it will be okay?

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy.signal import butter, sosfilt

def calculate_octave_bands_spl(file_path, target_sample_rate=22000):
    """
    Calculate SPL levels across 10 octave bands for a given audio file.
    Args:
        file_path (str): Path to the audio file.
        target_sample_rate (int): Target sample rate for processing.
    Returns:
        tuple: SPL levels for 10 octave bands, list of top 3 SPL peaks with their corresponding frequencies.
    """
    # Load the audio file
    sample_rate, data = wavfile.read(file_path)
    
    # Ensure data is mono
    if len(data.shape) == 2:
        data = np.mean(data, axis=1)
    
    # Downsample if necessary
    if sample_rate != target_sample_rate:
        num_samples = int(len(data) * target_sample_rate / sample_rate)
        data = np.interp(
            np.linspace(0, len(data), num_samples, endpoint=False),
            np.arange(len(data)),
            data
        )
        sample_rate = target_sample_rate

    # Define octave band center frequencies
    f_centers = [31.25, 62.5, 125, 250, 500, 1000, 2000, 4000, 8000, 16000]
    f_lower = [f / np.sqrt(2) for f in f_centers]
    f_upper = [f * np.sqrt(2) for f in f_centers]
    
    spl_values = []
    
    # Calculate SPL for each octave band
    for low, high in zip(f_lower, f_upper):
        nyquist = sample_rate / 2
        low_norm = low / nyquist
        high_norm = high / nyquist
        
        if low_norm <= 0 or high_norm >= 1:
            spl_values.append(-np.inf)  # Ignore out-of-range bands
            continue
        
        sos = butter(6, [low_norm, high_norm], btype='band', output='sos')
        filtered_signal = sosfilt(sos, data)
        rms = np.sqrt(np.mean(filtered_signal**2))
        spl = 20 * np.log10(rms) if rms > 0 else -np.inf
        spl_values.append(spl)
    
    # Identify the top 3 SPL peaks and their frequencies
    top_spl_indices = np.argsort(spl_values)[-3:][::-1]  # Indices of top 3 SPL values
    top_spl = [spl_values[i] for i in top_spl_indices]
    top_freqs = [f_centers[i] for i in top_spl_indices]
    
    return spl_values, top_spl, top_freqs

def process_audio_directory(input_directory, output_csv):
    """
    Process all audio files in a directory, calculate SPL levels and top 3 peaks.
    Args:
        input_directory (str): Path to the directory with preprocessed audio files.
        output_csv (str): Path to save the output CSV.
    """
    results = []
    
    for file_name in os.listdir(input_directory):
        if file_name.endswith(".wav"):
            file_path = os.path.join(input_directory, file_name)
            try:
                spl_levels, top_spl, top_freqs = calculate_octave_bands_spl(file_path)
                result_entry = {
                    "file": os.path.basename(file_path).replace('.wav', '')
                }
                # Add SPL levels for all bands
                for i, spl in enumerate(spl_levels):
                    result_entry[f"SPL_Band_{i+1}"] = spl
                
                # Add top 3 SPL peaks and their frequencies
                for i, (spl, freq) in enumerate(zip(top_spl, top_freqs)):
                    result_entry[f"Top_SPL_{i+1}"] = spl
                    result_entry[f"Top_Freq_{i+1}"] = freq
                
                results.append(result_entry)
            except Exception as e:
                print(f"Error processing {file_name}: {e}")
    
    # Save results to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"Processed data saved to {output_csv}")

# Define input and output paths
input_directory = "/Users/jakob/Downloads/IDMT_Traffic/preprocessed_audio_not_normalized"
output_csv = "octave_band_analysis_results.csv"

# Process the dataset
process_audio_directory(input_directory, output_csv)

Processed data saved to octave_band_analysis_results.csv


In [ ]:
# reminder to drop bands 9 and 10 from the df because of the -inf values

In [3]:
df = pd.read_csv("octave_band_analysis_results.csv")
df.head()


,file,SPL_Band_1,SPL_Band_2,SPL_Band_3,SPL_Band_4,SPL_Band_5,SPL_Band_6,SPL_Band_7,SPL_Band_8,SPL_Band_9,SPL_Band_10,Top_SPL_1,Top_Freq_1,Top_SPL_2,Top_Freq_2,Top_SPL_3,Top_Freq_3
0,2019-11-18-07-25_Langewiesener-Strasse_50Kmh_1...,-73.925254,-65.897689,-68.529411,-74.623105,-74.263679,-73.602205,-71.655332,-73.254970,-inf,-inf,-65.897689,62.50,-68.529411,125.0,-71.655332,2000.0
1,2019-11-19-07-25_Langewiesener-Strasse_50Kmh_1...,-66.726372,-48.161353,-49.223928,-43.817328,-39.337274,-37.541956,-41.328194,-47.518073,-inf,-inf,-37.541956,1000.00,-39.337274,500.0,-41.328194,2000.0
2,2019-10-22-15-30_Fraunhofer-IDMT_30Kmh_9062400...,-64.846447,-69.695176,-76.300927,-82.359539,-86.044668,-87.940355,-90.528599,-92.107125,-inf,-inf,-64.846447,31.25,-69.695176,62.5,-76.300927,125.0
3,2019-11-19-15-25_Langewiesener-Strasse_50Kmh_3...,-52.956888,-54.554779,-48.626704,-43.708284,-42.065760,-36.263034,-40.915098,-50.560191,-inf,-inf,-36.263034,1000.00,-40.915098,2000.0,-42.065760,500.0
4,2019-11-12-09-00_Schleusinger-Allee_70Kmh_3081...,-72.904765,-66.367636,-64.851481,-71.678644,-69.746037,-71.134836,-70.023266,-74.254897,-inf,-inf,-64.851481,125.00,-66.367636,62.5,-69.746037,500.0


# Zweiter Versuch: dieses Mal mit Code aus dem Github Projekt https://github.com/jmrplens/PyOctaveBand und angepasst auf unseren Use Case

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy.signal import butter, sosfilt, resample

def calculate_octave_bands_spl(file_path, limits=[12, 20000], fraction=1, order=6):
    """
    Calculate SPL levels across octave bands for a given audio file using Butterworth filters.
    Args:
        file_path (str): Path to the audio file.
        limits (list): Minimum and maximum frequency limits for filtering.
        fraction (int): Fraction of octave bands (e.g., 1 for 1-octave bands, 3 for 1/3-octave bands).
        order (int): Filter order for Butterworth filters.
    Returns:
        tuple: SPL levels for octave bands, list of top 3 SPL peaks with their corresponding frequencies.
    """
    # Load the audio file
    sample_rate, data = wavfile.read(file_path)
    
    # Ensure data is mono
    if len(data.shape) == 2:
        data = np.mean(data, axis=1)
    
    # Get standardized ANSI/IEC octave band frequencies and boundaries
    freq, freq_d, freq_u = getansifrequencies(fraction, limits)

    # Calculate downsampling factors for each band
    downsampling_factors = _downsamplingfactor(freq_u, sample_rate)

    # Prepare to store SPL values
    spl_values = []

    # Filter and calculate SPL for each band
    for idx, (low, high) in enumerate(zip(freq_d, freq_u)):
        # Downsample the signal
        downsample_factor = downsampling_factors[idx]
        if downsample_factor > 1:
            data_downsampled = resample(data, len(data) // downsample_factor)
            sample_rate_downsampled = sample_rate // downsample_factor
        else:
            data_downsampled = data
            sample_rate_downsampled = sample_rate

        # Normalize band frequencies for filtering
        low_norm = low / (sample_rate_downsampled / 2)
        high_norm = high / (sample_rate_downsampled / 2)
        
        if low_norm <= 0 or high_norm >= 1:
            spl_values.append(-np.inf)  # Ignore out-of-range bands
            continue

        # Create and apply bandpass filter
        sos = butter(order, [low_norm, high_norm], btype='bandpass', output='sos')
        filtered_signal = sosfilt(sos, data_downsampled)

        # Calculate SPL using standard deviation
        spl = 20 * np.log10(np.std(filtered_signal) / 2e-5) if np.std(filtered_signal) > 0 else -np.inf
        spl_values.append(spl)
    
    # Identify the top 3 SPL peaks and their frequencies
    top_spl_indices = np.argsort(spl_values)[-3:][::-1]  # Indices of top 3 SPL values
    top_spl = [spl_values[i] for i in top_spl_indices]
    top_freqs = [freq[i] for i in top_spl_indices]
    
    return spl_values, top_spl, top_freqs

def process_audio_directory(input_directory, output_csv, limits=[12, 20000], fraction=1, order=6):
    """
    Process all audio files in a directory, calculate SPL levels and top 3 peaks.
    Args:
        input_directory (str): Path to the directory with audio files.
        output_csv (str): Path to save the output CSV.
        limits (list): Minimum and maximum frequency limits for filtering.
        fraction (int): Fraction of octave bands (e.g., 1 for 1-octave bands, 3 for 1/3-octave bands).
        order (int): Filter order for Butterworth filters.
    """
    results = []
    
    for file_name in os.listdir(input_directory):
        if file_name.endswith(".wav"):
            file_path = os.path.join(input_directory, file_name)
            try:
                spl_levels, top_spl, top_freqs = calculate_octave_bands_spl(file_path, limits, fraction, order)
                result_entry = {
                    "file": os.path.basename(file_path).replace('.wav', '')
                }
                # Add SPL levels for all bands
                for i, spl in enumerate(spl_levels):
                    result_entry[f"SPL_Band_{i+1}"] = spl
                
                # Add top 3 SPL peaks and their frequencies
                for i, (spl, freq) in enumerate(zip(top_spl, top_freqs)):
                    result_entry[f"Top_SPL_{i+1}"] = spl
                    result_entry[f"Top_Freq_{i+1}"] = freq
                
                results.append(result_entry)
            except Exception as e:
                print(f"Error processing {file_name}: {e}")
    
    # Save results to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"Processed data saved to {output_csv}")

# Helper functions from your colleague's code
def getansifrequencies(fraction, limits):
    """ Generate standardized ANSI/IEC octave band frequencies and boundaries. """
    predefined = {1: _oneoctave(), 3: _thirdoctave()}
    freqs = predefined[fraction]
    freqs = [f for f in freqs if limits[0] <= f <= limits[1]]
    freq_d = [f / 2 ** (1 / (2 * fraction)) for f in freqs]
    freq_u = [f * 2 ** (1 / (2 * fraction)) for f in freqs]
    return freqs, freq_d, freq_u

def _downsamplingfactor(freq_u, fs):
    """ Calculate the downsampling factor for each band. """
    guard = 0.1
    return [max(min(int(fs / (2 * (f + guard))), 50), 1) for f in freq_u]

def _oneoctave():
    return [16, 31.5, 63, 125, 250, 500, 1000, 2000, 4000, 8000, 16000]

def _thirdoctave():
    return [12.5, 16, 20, 25, 31.5, 40, 50, 63, 80, 100, 125, 160, 200, 250,
            315, 400, 500, 630, 800, 1000, 1250, 1600, 2000, 2500, 3150, 4000,
            5000, 6300, 8000, 10000, 12500, 16000, 20000]

# Define input and output paths
input_directory = "/Users/jakob/Downloads/IDMT_Traffic/preprocessed_audio_not_normalized"
output_csv = "octave_band_analysis_v2.csv"

# Process the dataset
process_audio_directory(input_directory, output_csv)

Processed data saved to octave_band_analysis_v2.csv


### Merge datasets

In [4]:
import pandas as pd

# Load the existing dataset from Excel
df_dataset = pd.read_excel("df_dataset.xlsx")

# Load the new dataset from CSV
df_new = pd.read_csv("octave_band_analysis_v2.csv")

# Load mfcc_features_scipy.csv
df_mfcc = pd.read_csv("mfcc_features_scipy.csv")

# Merge the datasets on the "file" column
df_merged1 = pd.merge(df_dataset, df_mfcc, on="file")

# Merge the datasets on the "file" column
df_merged2 = pd.merge(df_merged1, df_new, on="file")

# Save the merged dataset to a new CSV file
df_merged2.to_csv("feature_set_spl_correct.csv", index=False)